In [17]:
import pandas as pd
from thunderpack import ThunderReader  # install thunderpack if not yet: pip install thunderpack
import pandas as pd
import random
import zipfile
import io
import tempfile
from tqdm import tqdm

In [2]:
#This part is importing the metadata into dfs
df_0 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2010_metadata.csv')
df_1 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2011_metadata.csv')
df_2 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2012_metadata.csv')
df_3 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2013_metadata.csv')
df_4 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2014_metadata.csv')
df_5 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2015_metadata.csv')
df_6 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2016_metadata.csv')
df_7 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2017_metadata.csv')
df_8 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2018_metadata.csv')
df_9 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2019_metadata.csv')
df_10 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2020_metadata.csv')
df_11 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2021_metadata.csv')
df_12 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2022_metadata.csv')
df_13 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2023_metadata.csv')
df_14 = pd.read_csv('/media/gregory178/Thunderpacks/Dropbox/zz_EHR_Thunderpacks/BIDMC/BIDMC_Deidentified_Notes_March14th2024/bidmc_notes_2024_metadata.csv')

dfs = pd.concat([df_0,df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10, df_11, df_12, df_13, df_14], ignore_index=True)  # List of DataFrames
print(len(dfs))
dfs.head()

25381614


,BDSPPatientID,NoteTypeFull,Service,CreateDate,DeidentifiedName
0,150641617,Letter,General Medicine/Primary Care,20100603,Notes_1130500157_1156166680_20100603.txt
1,150641617,Progress Note,General Medicine/Primary Care,20100812,Notes_1130500157_1156166681_20100812.txt
2,150641617,Progress Note,Emergency,20100813,Notes_1130500157_1156166682_20100813.txt
3,150641617,Initial Note,Psychiatry,20100813,Notes_1130500157_1156166683_20100813.txt
4,150641617,Telephone,Emergency,20100813,Notes_1130500157_1156166684_20100813.txt


In [14]:
# data filtering of note metadata
#changing date to datetime format
dfs['CreateDate'] = pd.to_datetime(dfs['CreateDate'], format='%Y%m%d')

# post epic only data
start_date = pd.Timestamp('2010-01-01')
end_date = pd.Timestamp('2024-12-31') #We should use these dates too. Only EPIC for MGB. 
complete_df_n = dfs[(dfs['CreateDate'] >= start_date) & (dfs['CreateDate'] < end_date)].reset_index(drop=True)

# filter all notes for initial notes only
complete_df_initial = complete_df_n[complete_df_n['NoteTypeFull'].str.lower() == 'initial note'].reset_index(drop=True)

print(len(complete_df_initial))
complete_df_initial.head()
complete_df_initial.to_csv("complete_df_initial.csv", index=False)


1981750


In [6]:
complete_df_initial=pd.read_csv("complete_df_initial.csv")

In [15]:
# Read icdPlus data and convert AdmissionDate to datetime
icdPlus = pd.read_csv('/home/gregory178/Desktop/NAX project/NAX_dive/patientIDs_ICD_plus_SDH_BI.csv')
icdPlus['AdmissionDate'] = pd.to_datetime(icdPlus['AdmissionDate'], format='%Y-%m-%d')

# Ensure BDSPPatientID is converted to int64 if needed
icdPlus['BDSPPatientID'] = icdPlus['BDSPPatientID'].astype('int64')

# Assuming complete_df_initial is already defined and contains your initial notes data
complete_df_initial['CreateDate'] = pd.to_datetime(complete_df_initial['CreateDate'], errors='coerce')

# Check for any dates that were not parsed correctly
invalid_dates = complete_df_initial[complete_df_initial['CreateDate'].isnull()]
if not invalid_dates.empty:
    print(f"Warning: Some dates in CreateDate could not be parsed:\n{invalid_dates}")

# Merge complete_df_initial with icdPlus on BDSPPatientID
final_cpt_pos_notes = pd.merge(complete_df_initial, icdPlus, on='BDSPPatientID', how='inner')

# Calculate the absolute difference in days between CreateDate and AdmissionDate
date_difference = abs((final_cpt_pos_notes['CreateDate'] - final_cpt_pos_notes['AdmissionDate']).dt.days)

# Filter the DataFrame to include only rows where the difference is less than 30 days
ICDPositiveDS = final_cpt_pos_notes[date_difference < 30].reset_index(drop=True)

# Save the filtered DataFrame to CSV
ICDPositiveDS.to_csv("BI_ICD_Positive_DS_before_500.csv", index=False)

In [14]:
ICDPositiveDS = pd.read_csv('/home/gregory178/Desktop/NAX project/NAX_dive/BI_ICD_Positive_DS_before_500.csv')

In [3]:
# run haoqi's code to get text and filter text for these notes
def select_random_note_zip(df, patient_id, base_zip_path):
    # Given patient ID read the notes from the zip file and then select one random note from notes longer than 500 words. 

    # df is the meta file with "BDSPPatientID", "ContactDate", "FileName"
    # patient_notes is the subset within df, for this patient only
    patient_notes = df[df['BDSPPatientID'] == patient_id]#patient notes is the metadata for this patient 
    valid_notes = []
    zip_file = None

    for _, row in patient_notes.iterrows():
        year = row['DeidentifiedName'].split('_')[-1][:4]
        file_path = f'{year}/' + row['DeidentifiedName']
        zip_path = f'{base_zip_path}bidmc_notes_{year}.zip'
        try:
            if zip_file is None or zip_file.filename != zip_path:
                if zip_file is not None:
                    zip_file.close()
                zip_file = zipfile.ZipFile(zip_path, 'r')
            
            with zip_file.open(file_path) as file_obj:
                try:
                    contents = file_obj.read().decode('utf-8')
                except UnicodeDecodeError:
                    try:
                        contents = file_obj.read().decode('latin1')
                    except UnicodeDecodeError:
                        contents = file_obj.read().decode('iso-8859-1')
                #Here we sort through based on word count. 
                word_count = len(contents.split())
                print(f"Length of contents for {file_path}: {word_count}")
                if word_count >= 500:
                    valid_notes.append((row['BDSPPatientID'], contents, row['CreateDate'], word_count, row['DeidentifiedName'], 'BIDMC'))
        except (KeyError, FileNotFoundError, zipfile.BadZipFile) as e:
            print(f"Error processing file {file_path}: {str(e)}")
            continue
    if zip_file is not None:
        zip_file.close()
    if valid_notes:
        selected_note = random.choice(valid_notes) #Randomly select one note from notes longer than 500 words.
        return selected_note
    else:
        return None

base_zip_path= '/media/gregory178/Elements/Zip_Files_BI/'
ICDPL_df=pd.read_csv('/home/gregory178/Desktop/NAX project/NAX_dive/BI_ICD_Positive_DS_before_500.csv')
FULLICDPL_list = ICDPL_df['BDSPPatientID'].tolist()  # this is the list of all patients with at least one ICD code

# ICDMI_df=pd.read_csv('patientIDs_ICD_minus_SDH_MGB.csv')
# FULLICDMI_list = ICDMI_df['BDSPPatientID'].tolist()

#IS HE MAKING A TABLE HERE?
columns = ['BDSPPatientID', 'note_text', 'CreateDate', 'length', 'deidentifiedname', 'hospital']

#Making ICD + and - groups for MGB
ICDPL_BI = pd.DataFrame()
ICDMI_BI = pd.DataFrame()



In [7]:
# now we are doing ICD+ and discharge notes
# output ++ group to a csv
counterPL = 0
added_patient_idsPL = []
print(len(added_patient_idsPL))
print(added_patient_idsPL)

# to ensure the reproducibility
random.seed(2024)

while len(ICDPL_BI) < 750:
    randpatientID = random.choice(FULLICDPL_list) # this means to select one patient randomly, however, it makes it not reproducible
    UPDATED_LEN_GROUP=len(FULLICDPL_list)
    print(f"Updated length list {UPDATED_LEN_GROUP}.")
    if randpatientID not in added_patient_idsPL:
        rnote2 = select_random_note_zip(complete_df_initial, randpatientID, base_zip_path)
        if rnote2 is not None:
            ICDPL_BI = pd.concat([ICDPL_BI, pd.DataFrame([rnote2])], ignore_index=True) #, columns=columns
            added_patient_idsPL.append(randpatientID)
            FULLICDPL_list.remove(randpatientID)
            counterPL += 1
            print(f"Added note for patient {randpatientID}. Total notes added: {counterPL}")
        else:
            print(f"No valid notes for patient {randpatientID}")
            FULLICDPL_list.remove(randpatientID)
    else:
        print(f"Patient {randpatientID} already has a note added")
        FULLICDPL_list.remove(randpatientID)
    if not FULLICDPL_list:
        print("No more patient IDs left to process")

ICDPL_BI.columns = ['BDSPPatientID', 'NoteContent', 'ContactDate', 'WordCount', 'NoteFileName', 'Site']
ICDPL_BI.to_csv('BI_sampling_cohort_ICD+_initial_notes.csv', index=False)

0
[]
Updated length list 48834.
Length of contents for 2013/Notes_1130542094_11409543334_20130117.txt: 1191
Length of contents for 2015/Notes_1130542094_11409543380_20150928.txt: 1329
Length of contents for 2015/Notes_1130542094_11409543391_20151212.txt: 427
Length of contents for 2016/Notes_1130542094_11409543407_20160330.txt: 1659
Length of contents for 2016/Notes_1130542094_11409543409_20160331.txt: 175
Length of contents for 2016/Notes_1130542094_11409543418_20160504.txt: 2380
Length of contents for 2016/Notes_1130542094_11409543434_20160601.txt: 1370
Length of contents for 2016/Notes_1130542094_11409543435_20160601.txt: 1227
Length of contents for 2018/Notes_1130542094_11409543450_20180425.txt: 1308
Length of contents for 2018/Notes_1130542094_11409543452_20180425.txt: 557
Length of contents for 2018/Notes_1130542094_11409543494_20180711.txt: 1577
Length of contents for 2018/Notes_1130542094_11409543497_20180712.txt: 249
Length of contents for 2018/Notes_1130542094_11409543507_201

In [12]:
### create the minus sampling cohort

# the minus cohort can be defiend as the difference between the whole dataset and the plus cohort
# and then, we will limit to patients with notes with >=500 words

# run the upper block to generates complete_df_discharge, if you haven't done it

# we want to further limit the full set to people without ICD (ICD-)
icd_minus_df = pd.read_csv('patientIDs_ICD_minus2_SDH_BI.csv')
full_patient_list = sorted( set(complete_df_initial.BDSPPatientID) & set(icd_minus_df.BDSPPatientID) )

# FULLICDMI_list = full_patient_list - FULLICDPL_list
# method 1:
#FULLICDMI_list = [x for x in full_patient_list if x not in FULLICDPL_list]

# method 2:
FULLICDMI_list = sorted( set(full_patient_list) - set(FULLICDPL_list) )   # <-- Haoqi prefers this way, since the result has unique patientIDs.

In [13]:
### now we are doing ICD- and discharge notes

counterMI = 0
added_patient_idsMI = []
print(len(added_patient_idsMI))
print(added_patient_idsMI)
random.seed(13)

while len(ICDMI_BI) < 750:
    randpatientID = random.choice(FULLICDMI_list) # this means to select one patient randomly, however, it makes it not reproducible
    UPDATED_LEN_GROUP=len(FULLICDMI_list)
    print(f"Updated length list {UPDATED_LEN_GROUP}.")
    if randpatientID not in added_patient_idsMI:
        rnote2 = select_random_note_zip(complete_df_initial, randpatientID, base_zip_path)
        if rnote2 is not None:
            ICDMI_BI = pd.concat([ICDMI_BI, pd.DataFrame([rnote2])], ignore_index=True) #, columns=columns
            added_patient_idsMI.append(randpatientID)
            FULLICDMI_list.remove(randpatientID)
            counterMI += 1
            print(f"Added note for patient {randpatientID}. Total notes added: {counterMI}")
        else:
            print(f"No valid notes for patient {randpatientID}")
            FULLICDMI_list.remove(randpatientID)
    else:
        print(f"Patient {randpatientID} already has a note added")
        FULLICDMI_list.remove(randpatientID)
    if not FULLICDMI_list:
        print("No more patient IDs left to process")

ICDMI_BI.columns = ['BDSPPatientID', 'NoteContent', 'ContactDate', 'WordCount', 'NoteFileName', 'Site']
ICDMI_BI.to_csv('BI_sampling_cohort_ICD_minus_initial_notes.csv', index=False)

0
[]
Updated length list 253415.
Length of contents for 2016/Notes_1130348156_199354442_20161028.txt: 1802
Length of contents for 2017/Notes_1130348156_199354450_20170218.txt: 68
Added note for patient 150489541. Total notes added: 1
Updated length list 253414.
Length of contents for 2012/Notes_1131012655_414145045_20120108.txt: 587
Added note for patient 151154144. Total notes added: 2
Updated length list 253413.
Length of contents for 2019/Notes_1130995807_408200447_20190531.txt: 832
Added note for patient 151136744. Total notes added: 3
Updated length list 253412.
Length of contents for 2011/Notes_1131002599_410611545_20110420.txt: 1558
Added note for patient 151144144. Total notes added: 4
Updated length list 253411.
Length of contents for 2016/Notes_1131106908_450429808_20161215.txt: 601
Length of contents for 2016/Notes_1131106908_450429810_20161222.txt: 446
Length of contents for 2017/Notes_1131106908_3143064220_20170209.txt: 1413
Length of contents for 2017/Notes_1131106908_300